In [2]:
import datetime as dt
import matplotlib as mp
import numpy as np
from data_wrangling import *
from analytics import *
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
import lightgbm as lgb

pd.set_option('display.max_columns', 150)
# documentation: https://support.google.com/analytics/answer/3437719?hl=en
# Note: this field should be unique, but it is not (around 1800 repeated unique_visit_id's)
#trainCSV['unique_visit_id'] = trainCSV.fullVisitorId +'_'+ trainCSV.visitId.apply(str)

cool_columns = ['fullVisitorId', 'date', 'geoNetwork_city', 'geoNetwork_country', 'device_browser','visits', 'paying_visits', 'totals_transactionRevenue']


In [ ]:
%%time

trainCSV = pd.read_csv('train_v2_flat.csv', sep=',', encoding='utf-8', low_memory=False).drop(columns=['trafficSource_campaignCode'])
trainCSV = fill_empty_values(trainCSV)
#trainCSV = trainCSV[cool_columns]

#testCSV  = pd.read_csv('test_v2_flat.csv', sep=',', encoding='utf-8', low_memory=False)
#testCSV = fill_empty_values(testCSV)

start_train_visits = trainCSV.date.min()
end_train_visits = trainCSV.date.max() - timedelta(days=276)
train_visits = trainCSV[(trainCSV.date >= start_train_visits) & (trainCSV.date<end_train_visits)]
validation_visits = trainCSV[(trainCSV.date>= end_train_visits)]
#test_visits = testCSV

In [ ]:
%%time
train = get_training_data_from_visits(train_visits)
validation = get_training_data_from_visits(validation_visits)

In [ ]:
%%time
train_e = label_encode_object_dtypes(encoder_trainer = train, df_to_encode = train)
validation_e = label_encode_object_dtypes(encoder_trainer = train, df_to_encode = validation)

In [ ]:
#train_e.to_csv("train_all_columns.csv", index=False)
#validation_e.to_csv("validation_all_columns.csv", index=False)

In [3]:
%%time
train_data = pd.read_csv("train_all_columns.csv")
validation_data = pd.read_csv("validation_all_columns.csv")

X_train = train_data.drop(columns=['target'])
y_train = train_data.target

X_validation = validation_data.drop(columns=['target'])
y_validation = validation_data.target

CPU times: user 29 s, sys: 2.21 s, total: 31.2 s
Wall time: 31.3 s


In [17]:
#### # create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_validation, y_validation, reference=lgb_train)

# specify your configurations as a dict
params = {
    'num_leaves': 50,
    'min_data_in_leaf': 100,
    'max_depth': 30,
    'metric': {'l2', 'l1'}
    #'boosting_type': 'gbdt',
    #'objective': 'regression',
    
    #'num_leaves': 31,
    #'learning_rate': 0.05,
    #'feature_fraction': 0.9,
    #'bagging_fraction': 0.8,
    #'bagging_freq': 5,
    #'verbose': 0
}

print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

print('Saving model...')
# save model to file
gbm.save_model('model.txt')

print('Starting predicting...')
# predict
y_pred = gbm.predict(X_validation, num_iteration=gbm.best_iteration)
# eval
print('The rmse of prediction is:', mean_squared_error(y_validation, y_pred) ** 0.5)




Starting training...
[1]	valid_0's l1: 0.0129708	valid_0's l2: 0.108423
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l1: 0.012921	valid_0's l2: 0.10832
[3]	valid_0's l1: 0.012915	valid_0's l2: 0.108248
[4]	valid_0's l1: 0.0128745	valid_0's l2: 0.108204
[5]	valid_0's l1: 0.0128615	valid_0's l2: 0.108211
[6]	valid_0's l1: 0.012806	valid_0's l2: 0.1083
[7]	valid_0's l1: 0.0127846	valid_0's l2: 0.108364
[8]	valid_0's l1: 0.0127576	valid_0's l2: 0.108314
[9]	valid_0's l1: 0.0127436	valid_0's l2: 0.108304
Early stopping, best iteration is:
[4]	valid_0's l1: 0.0128745	valid_0's l2: 0.108204
Saving model...
Starting predicting...
('The rmse of prediction is:', 0.32894348844984)


In [ ]:
y_predicted = regr.predict(X_validation)
print "benchmark_mse", mean_squared_error(y_validation, y_predicted)
print "benchmark_rmse",  np.sqrt(mean_squared_error(y_validation, y_predicted))

In [5]:
y_all_zeros = [0]*len(y_validation)
y_predicted = y_all_zeros
print "benchmark_mse", mean_squared_error(y_validation, y_all_zeros)
print "benchmark_rmse",  np.sqrt(mean_squared_error(y_validation, y_all_zeros))

benchmark_mse 0.10863334141746757
benchmark_rmse 0.329595724209929
